# MelodyFlow
Welcome to MelodyFlow's demo jupyter notebook. 
Here you will find a self-contained example of how to use MelodyFlow for music generation and editing.

First, we start by initializing MelodyFlow for music generation, you can choose a model from the following selection:
1. facebook/melodyflow-t24-30secs - 1B parameters, 30 seconds music samples.

We will use the `facebook/melodyflow-t24-30secs` variant for the purpose of this demonstration.

In [ ]:
from audiocraft.models import MelodyFlow

model = MelodyFlow.get_pretrained("facebook/melodyflow-t24-30secs")

Next, let us configure the generation parameters. Specifically, you can control the following:
* `solver` (str, optional): ODE solver, either euler or midpoint. Defaults to midpoint.
* `steps` (int, optional): number of solver steps. Defaults to 64.

When left unchanged, MelodyFlow will revert to its default parameters.

In [3]:
model.set_generation_params(
    solver="midpoint",
    steps=64,
)

Next, we can go ahead and start generating music given textual prompts.

### Text-conditional Generation

In [ ]:
from audiocraft.utils.notebook import display_audio

###### Text-to-music prompts - examples ######
text = "80s electronic track with melodic synthesizers, catchy beat and groovy bass"
# text = "80s electronic track with melodic synthesizers, catchy beat and groovy bass. 170 bpm"
# text = "Earthy tones, environmentally conscious, ukulele-infused, harmonic, breezy, easygoing, organic instrumentation, gentle grooves"
# text = "Funky groove with electric piano playing blue chords rhythmically"
# text = "Rock with saturated guitars, a heavy bass line and crazy drum break and fills."
# text = "A grand orchestral arrangement with thunderous percussion, epic brass fanfares, and soaring strings, creating a cinematic atmosphere fit for a heroic battle"
                   
N_VARIATIONS = 3
descriptions = [text for _ in range(N_VARIATIONS)]

print(f"text prompt: {text}\n")
output = model.generate(descriptions=descriptions, progress=True, return_tokens=True)
display_audio(output[0], sample_rate=model.compression_model.sample_rate)

### Text-conditional Editing

Next, let us configure the editing parameters. Specifically, you can control the following:
* `solver` (str, optional): ODE solver, either euler or midpoint. Defaults to euler.
* `steps` (int, optional): number of solver steps. Defaults to 25.
* `target_flowstep` (float, optional): Target flow step. Defaults to 0.
* `regularize` (int, optional): Regularize each solver step. Defaults to True.
* `regularize_iters` (int, optional): Number of regularization iterations. Defaults to 4.
* `keep_last_k_iters` (int, optional): Number of meaningful regularization iterations for moving average computation. Defaults to 2.
* `lambda_kl` (float, optional): KL regularization loss weight. Defaults to 0.2.

When left unchanged, MelodyFlow will revert to its default parameters.

In [5]:
model.set_editing_params(
    solver = "euler",
    steps = 25,
    target_flowstep = 0.05,
    regularize = True,
    regularize_iters = 4,
    keep_last_k_iters = 2,
    lambda_kl = 0.2,
)

Next, we can go ahead and edit the previously generated music given new textual prompts.

In [ ]:
edited_output = model.edit(output[1],
                        ["Piano melody." for _ in range(N_VARIATIONS)],
                        src_descriptions=["" for _ in range(N_VARIATIONS)],
                        return_tokens=True,
                        progress=True)
display_audio(edited_output[0], sample_rate=model.compression_model.sample_rate)